### Inicio de limpieza sencilla de datos a partir del archivo general_11_24_ev_renta_demog_paro_pobl_zv_obras_unificado.csv  ###


In [ ]:
pip install unidecode

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
import numpy as np
from unidecode import unidecode

# Cargar el dataset original desde CSV
df = pd.read_csv("general_11_24_ev_renta_demog_paro_pobl_zv_obras_unificado.csv",
    na_values=["", " ", "NA", "N/A", "null"]  # interpreta como nulos
)

# Eliminar la columna 'Unnamed: 0' 
if "Unnamed: 0" in df.columns:
    df = df.drop(columns=["Unnamed: 0"])

# Convertir espacios vacios en NaN
df = df.replace(r'^\s*$', np.nan, regex=True)

# Normalizar nombres de columnas
df.columns = [unidecode(col).replace(" ", "_") for col in df.columns]

# Revisar nombres finales de columnas
print("Columnas después de limpieza:\n", df.columns)

# Revisar nulos por columna (opcional)
print("\nNulos por columna:\n", df.isna().sum())

df.to_csv("general_11_24_ev_renta_demog_paro_pobl_zv_obras_clean.csv", index=False)


C:\Users\andre\AppData\Local\Temp\ipykernel_19952\2570342329.py:6: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("general_11_24_ev_renta_demog_paro_pobl_zv_obras_unificado.csv",


Columnas después de limpieza:
 Index(['Distrito', 'Barrio', 'anyo', 'Inversion', 'airbnb', 'Superficie_(Ha.)',
       'Poblacion_densidad_(hab./Ha.)', 'Tasa_bruta_de_natalidad_(%0)',
       'Poblacion_en_etapas_educativas',
       'Tasa_de_crecimiento_demografico_(porcentaje)',
       'Porcentaje_de_envejecimiento_(Poblacion_mayor_de_65_anos/Poblacion_total)',
       'Paro_registrado_(numero_de_personas_registradas_en_SEPE_en_febrero)',
       'Tamano_medio_del_hogar_x',
       'Valor_catastral_medio_por_inmueble_de_uso_residencial',
       'Hogares_con_un_hombre_solo_mayor_de_65_anos',
       'Hogares_con_una_mujer_sola_mayor_de_65_anos',
       'Edad_media_de_la_poblacion_x', 'Numero_Habitantes',
       'Poblacion_Mujeres', 'Total_hogares',
       'Personas_con_nacionalidad_espanola',
       'Personas_con_nacionalidad_extranjera',
       'Renta_neta_media_anual_de_los_hogares_(Urban_Audit)',
       'num_hogar_monoparental', 'anyo_timestamp', 'poblacion_noEstudios',
       'poblacion_

In [ ]:
# Eliminar columnas que no corresponden
# Se eliminó por "anyo_timestamp" porque tenia información de dia/mes/año y era demasiado granular, ademas tenia pocos datos 
# Se eliminó "poblacion_anual" porque esta columna se corrigió y se agregó nuevamente como "promedio_anual" para asegurar que fuera un promedio 
# Se elimino la columna "número_habitantes" porque teniamos información más nueva y con mas años, ver columna "Poblacion"
df = df.drop(columns=["anyo_timestamp", "poblacion_anual", "Numero_Habitantes"], errors="ignore")

# Revisar columnas después de eliminar
print("Columnas después de eliminar:\n", df.columns)

df.to_csv("general_11_24_ev_renta_demog_paro_pobl_zv_obras_clean.csv", index=False)


Columnas después de eliminar:
 Index(['Distrito', 'Barrio', 'anyo', 'Inversion', 'airbnb', 'Superficie_(Ha.)',
       'Poblacion_densidad_(hab./Ha.)', 'Tasa_bruta_de_natalidad_(%0)',
       'Poblacion_en_etapas_educativas',
       'Tasa_de_crecimiento_demografico_(porcentaje)',
       'Porcentaje_de_envejecimiento_(Poblacion_mayor_de_65_anos/Poblacion_total)',
       'Paro_registrado_(numero_de_personas_registradas_en_SEPE_en_febrero)',
       'Tamano_medio_del_hogar_x',
       'Valor_catastral_medio_por_inmueble_de_uso_residencial',
       'Hogares_con_un_hombre_solo_mayor_de_65_anos',
       'Hogares_con_una_mujer_sola_mayor_de_65_anos',
       'Edad_media_de_la_poblacion_x', 'Poblacion_Mujeres', 'Total_hogares',
       'Personas_con_nacionalidad_espanola',
       'Personas_con_nacionalidad_extranjera',
       'Renta_neta_media_anual_de_los_hogares_(Urban_Audit)',
       'num_hogar_monoparental', 'poblacion_noEstudios',
       'poblacion_estudios_medios', 'poblacion_estudios_superior

In [ ]:
import pandas as pd
import re

# Función para limpiar columnas numéricas en formato texto:
# elimina % , espacios y símbolos extra,
# cambia coma por punto y convierte a número válido

def clean_numeric_series(s: pd.Series) -> pd.Series:
    
    s = s.astype("string")
    s = s.str.replace(r"%", "", regex=True)
    s = s.str.replace(r"\s+", "", regex=True)
    s = s.str.replace(",", ".", regex=False)
    s = s.str.replace(r"[^0-9\.\-eE]", "", regex=True)
    return pd.to_numeric(s, errors="coerce")

# Carga CSV
df = pd.read_csv("general_11_24_ev_renta_demog_paro_pobl_zv_obras_clean.csv")

# Renombrar columnas
rename_map = {
    "Distrito": "Distrito",
    "Barrio": "Barrio",
    "anyo": "Anyo",
    "Inversion": "Inversiones",
    "airbnb": "Cantidad_airbnb",
    "Superficie_(Ha.)": "Superficie_ha",
    "Poblacion_densidad_(hab./Ha.)": "Densidad_pob/ha",
    "Tasa_bruta_de_natalidad_(%0)": "Natalidad%",
    "Poblacion_en_etapas_educativas": "Poblacion_etapa_educativa",
    "Tasa_de_crecimiento_demografico_(porcentaje)": "Crecimiento_demografico%",
    "Porcentaje_de_envejecimiento_(Poblacion_mayor_de_65_anos/Poblacion_total)": "Envejecimiento%",
    "Paro_registrado_(numero_de_personas_registradas_en_SEPE_en_febrero)": "Paro_registrado%",
    "Tamano_medio_del_hogar_x": "Tamano_vivienda_1",
    "Valor_catastral_medio_por_inmueble_de_uso_residencial": "Valor_catastral",
    "Hogares_con_un_hombre_solo_mayor_de_65_anos": "Hogar_hombres_mayores65",
    "Hogares_con_una_mujer_sola_mayor_de_65_anos": "Hogar_mujeres_mayores65",
    "Edad_media_de_la_poblacion_x": "Edad_media_1",
    "Poblacion_Mujeres": "Poblacion_mujeres",
    "Total_hogares": "Total_viviendas",
    "Personas_con_nacionalidad_espanola": "Poblacion_espanola",
    "Personas_con_nacionalidad_extranjera": "Poblacion_extranjera",
    "Renta_neta_media_anual_de_los_hogares_(Urban_Audit),": "Renta_media_hogares",
    "num_hogar_monoparental": "Hogar_monoparental",
    "poblacion_noEstudios": "Poblacion_noestudio",
    "poblacion_estudios_medios": "Poblacion_estudiomedio",
    "poblacion_estudios_superiores": "Poblacion_estudiosuperior",
    "precio_vivienda": "Precio_vivienda",
    "Precio_venta": "Precio_venta",          # se mantiene separada
    "Precio_alquiler": "Precio_alquiler",
    "esperanza_vida": "Esperanza_vida",
    "renta_media_persona": "Renta_media_persona",
    "Edad_media_de_la_poblacion_y": "Edad_media_2",
    "Poblacion": "Poblacion_anual_1",
    "Porcentaje_de_poblacion_de_65_y_mas_anos": "Mayores_65anyos%",
    "Porcentaje_de_poblacion_menor_de_18_anos": "Menores_18anyos%",
    "Tamano_medio_del_hogar_y": "Tamano_vivienda_2",
    "paro_registrado_anual": "Tasa_Paro",
    "ha_de_ZONAS_VERDES_Y_PARQUES_en_distrito": "Zonas_verdes_ha",
    "obras_iniciadas": "Obras_iniciadas",
    "obras_finalizadas": "Obras_finalizadas",
    "promedio_anual": "Poblacion_anual_2",
}

df = df.rename(columns=rename_map)

# Tipado de columnas
# Texto
string_cols = ["Distrito", "Barrio"]

# Enteros
int_cols = [
    "Anyo", "Poblacion_etapa_educativa", "Hogar_hombres_mayores65",
    "Hogar_mujeres_mayores65", "Poblacion_mujeres", "Total_viviendas",
    "Poblacion_espanola", "Poblacion_extranjera", "Hogar_monoparental",
    "Poblacion_noestudio", "Poblacion_estudiomedio", "Poblacion_estudiosuperior",
    "Obras_iniciadas", "Obras_finalizadas", "Poblacion_anual_1", "Poblacion_anual_2"
]

# Decimales
float_cols = [
    "Inversiones", "Cantidad_airbnb", "Superficie_ha", "Densidad_pob/ha",
    "Natalidad%", "Crecimiento_demografico%", "Envejecimiento%",
    "Paro_registrado%", "Tamano_vivienda_1", "Valor_catastral",
    "Edad_media_1", "Renta_media_hogares", "Precio_vivienda", "Precio_venta",
    "Precio_alquiler", "Esperanza_vida", "Renta_media_persona", "Edad_media_2",
    "Mayores_65anyos%", "Menores_18anyos%", "Tamano_vivienda_2",
    "Tasa_Paro", "Zonas_verdes_ha"
]

# Normalizar numéricos que puedan estar como texto
for col in float_cols:
    if col in df.columns:
        df[col] = clean_numeric_series(df[col]).astype("float64")

for col in int_cols:
    if col in df.columns:
        
        try:
            df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")
        except Exception:
            df[col] = clean_numeric_series(df[col]).astype("Int64")

# Texto
for col in string_cols:
    if col in df.columns:
        df[col] = df[col].astype("string")

df.to_csv("datos_madrid_unificado_type.csv", index=False)
print("Guardado: datos_madrid_unificado_std.csv")
print(df.dtypes)


✅ Guardado: datos_madrid_unificado_typed_1.csv
Distrito                                               string[python]
Barrio                                                 string[python]
Anyo                                                            Int64
Inversiones                                                   float64
Cantidad_airbnb                                               float64
Superficie_ha                                                 float64
Densidad_pob/ha                                               float64
Natalidad%                                                    float64
Poblacion_etapa_educativa                                       Int64
Crecimiento_demografico%                                      float64
Envejecimiento%                                               float64
Paro_registrado%                                              float64
Tamano_vivienda_1                                             float64
Valor_catastral                            